In [1]:
import requests
from lxml import html
from urllib.request import urlopen, Request

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

from bs4 import BeautifulSoup
import re
import pandas as pd
import os
import time
from datetime import datetime
import threading
from datetime import date
from apscheduler.schedulers.background import BackgroundScheduler

In [2]:
entry_point = "https://www.udacity.com"

In [3]:
logged_in = False

#setup option for chrome profile
chrome_options = Options()
# chrome_options.add_argument("user-data-dir=/Users/huohsien/Library/Application Support/Google/Chrome/Default/")

#start web driver
browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
browser.implicitly_wait(5)

browser.get(entry_point)

wait = WebDriverWait(browser, 5)

[WDM] - Current google-chrome version is 83.0.4103
[WDM] - Get LATEST driver version for 83.0.4103
[WDM] - Driver [/Users/huohsien/.wdm/drivers/chromedriver/mac64/83.0.4103.39/chromedriver] found in cache


In [4]:
#close pop up ads
try:
    cancel_button = wait.until(ec.visibility_of_element_located((By.XPATH, '//div[@class="modal-close dark"]')))
    ActionChains(browser).move_to_element(cancel_button).click().perform()
except:
  print("the ads popup does not show up")



In [5]:
try:
    link = browser.find_element_by_xpath("//a[@title='My Classroom']")
    print("link= ", link)
    logged_in = True

except:
    logged_in = False
    print("need to login")

if logged_in:
    print("already logged in")
    browser.execute_script("arguments[0].click();", link)

need to login


In [6]:
if not logged_in:
#     time.sleep(0.5)
    wait.until(ec.visibility_of_element_located((By.LINK_TEXT, 'Sign In')))
    signin_link = browser.find_element_by_link_text('Sign In')
    print("signin_link= ", signin_link)
    signin_link.get_attribute('href')
    signin_link.click()
#     time.sleep(0.5)
    # signin_button = wait.until(ec.visibility_of_element_located((By.XPATH, '//a[@title="Sign In"]')))
    # ActionChains(browser).move_to_element(signin_button).click().perform()

    wait.until(ec.visibility_of_element_located((By.XPATH, "//input[@data-testid='signin-email']")))
    email = browser.find_element_by_xpath("//input[@data-testid='signin-email']")
    wait.until(ec.visibility_of_element_located((By.XPATH, "//input[@data-testid='signin-password']")))
    password = browser.find_element_by_xpath("//input[@data-testid='signin-password']")

    email.send_keys("huohsien@gmail.com")
    password.send_keys("jj1216")

    wait.until(ec.visibility_of_element_located((By.CLASS_NAME, "button_primary__1qhjh.button__btn__2MHEg.form_primary-button__37eaW.button_standard__1p9sb")))
    sign_in_btn = browser.find_element_by_class_name("button_primary__1qhjh.button__btn__2MHEg.form_primary-button__37eaW.button_standard__1p9sb") 
    print("sign_in_btn= ", sign_in_btn)
    sign_in_btn.click()
    print("sign_in_btn clicked")


signin_link=  <selenium.webdriver.remote.webelement.WebElement (session="761c10dc82efd7a6f1544e713e5a0c1c", element="311c8154-76ae-4a50-8cde-4cdfb80640a3")>
sign_in_btn=  <selenium.webdriver.remote.webelement.WebElement (session="761c10dc82efd7a6f1544e713e5a0c1c", element="cf0ea6d6-609a-4252-a1e6-5b4ac6db8a7f")>
sign_in_btn clicked


In [7]:
wait.until(ec.presence_of_element_located((By.XPATH, "//a[@href='#nanodegrees']")))

print("href found")
button = browser.find_element_by_xpath("//a[@href='#nanodegrees']") 
print("button= ", button)
button.click()
print("button clicked")

href found
button=  <selenium.webdriver.remote.webelement.WebElement (session="761c10dc82efd7a6f1544e713e5a0c1c", element="996f6f21-583a-41e7-8a0d-a740815a79b3")>
button clicked


In [8]:
elms = browser.find_elements_by_xpath("//*[button='Deep Learning']")

entry = None
for elm in elms:
    text = elm.text
    if "Graduated" in text:
        entry = elm

button = entry.find_element_by_xpath("./div/div/button")
button.click()

# Start to navigate the core curriculum

In [9]:
def check_complete(item)->bool:
    e1 = item.find_element_by_xpath("./div/div")
    c = e1.get_attribute('class')
    if c == 'index--lesson-card--mwX1V index--card--3DZMr shared--card--3X88h':
        return True
    else:
        return False
#     print(c)

In [10]:
topics = browser.find_elements_by_xpath("//*[@id='tree-core-curriculum']/*")
for topic in topics[:1]:
    link = topic.find_element_by_xpath("./a")
    link.click()
    print("wait")
    wait.until(ec.visibility_of_element_located((By.CLASS_NAME, "nav.primary-and-secondary")))
    print("sub-topic page loaded")
    
    sections = browser.find_elements_by_xpath("//div[@data-test='part-syllabus']/ol/li")
    
    #click open the ones that were marked Complete
    for section in sections:
        if check_complete(section):
            section.click()
    sections = browser.find_elements_by_xpath("//div[@data-test='part-syllabus']/ol/li")
    
    for section in sections:
        print("section= ", section.text)
        section.click()
        wait.until(ec.visibility_of_element_located((By.CLASS_NAME, "_main--content-container--ILkoI")))
        print("sub-topic content page loaded")
        list = browser.find_elements_by_xpath("//ol[@class='index--contents-list--33vHB']/li")
        for item in list:
            item.click()
            time.sleep(5)


wait
sub-topic page loaded
section=  View
LESSON 1
Welcome to the Deep Learning Nanodegree Program
Welcome to the Deep Learning Nanodegree program! Learn about the program structure and resources!
VIEW LESSON
sub-topic content page loaded


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=83.0.4103.116)


In [ ]:
# sections = browser.find_elements_by_xpath("//a[@class='index--curtain--3sKxm.shared--curtain--2_FSP']")
# print(sections)
# for section in sections:
#     print(section.text)
#     print("-----")
